In [1]:
import pyspark
from pyspark.sql.functions import udf
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import MinMaxScaler
import pyspark.ml.feature as feat
import pyspark.ml.classification as cl
from pyspark.ml.feature import VectorAssembler

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1588797194727_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = (
spark
.read
.csv(
    's3://travis-grace-4567-final-project/ks-projects-201801.csv'
    ,header=True)
)

df.createOrReplaceTempView("df")
df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ID: string, name: string, category: string, main_category: string, currency: string, deadline: string, goal: string, launched: string, pledged: string, state: string, backers: string, country: string, usd_pledged: string, usd_pledged_real: string, usd_goal_real: string]

In [3]:
print(type(df))
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>
+----------+--------------------+--------------+-------------+--------+----------+------+---------------+-------+----------+-------+-------+-----------+----------------+-------------+
|        ID|                name|      category|main_category|currency|  deadline|  goal|       launched|pledged|     state|backers|country|usd_pledged|usd_pledged_real|usd_goal_real|
+----------+--------------------+--------------+-------------+--------+----------+------+---------------+-------+----------+-------+-------+-----------+----------------+-------------+
|1000002330|The Songs of Adel...|        Poetry|   Publishing|     GBP| 10/9/2015|  1000|8/11/2015 12:12|      0|    failed|      0|     GB|          0|               0|      1533.95|
|1000003930|Greeting From Ear...|Narrative Film| Film & Video|     USD| 11/1/2017| 30000|  9/2/2017 4:43|   2421|    failed|     15|     US|        100|            2421|        30000|
|1000004038|      Where is Hank?|Narra

In [4]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: string (nullable = true)
 |-- country: string (nullable = true)
 |-- usd_pledged: string (nullable = true)
 |-- usd_pledged_real: string (nullable = true)
 |-- usd_goal_real: string (nullable = true)

In [5]:
df = spark.sql("""
            select cast(f.ID as int)
            , f.name
            , f.category
            , f.main_category
            , f.currency
            , to_date(f.deadline, 'MM/dd/yy') as `deadline`
            , cast(f.goal as double)
            , to_date(f.launched, 'MM/dd/yy') as `launched`
            , cast(f.pledged as double)
            , f.state
            , cast(f.backers as int)
            , f.country
            , cast(f.usd_pledged as double)
            , cast(f.usd_pledged_real as double)
            , cast(f.usd_goal_real as double)
            from df as f
            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
print(type(df))
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>
+----------+--------------------+--------------+-------------+--------+----------+--------+----------+-------+----------+-------+-------+-----------+----------------+-------------+
|        ID|                name|      category|main_category|currency|  deadline|    goal|  launched|pledged|     state|backers|country|usd_pledged|usd_pledged_real|usd_goal_real|
+----------+--------------------+--------------+-------------+--------+----------+--------+----------+-------+----------+-------+-------+-----------+----------------+-------------+
|1000002330|The Songs of Adel...|        Poetry|   Publishing|     GBP|2015-10-09|  1000.0|2015-08-11|    0.0|    failed|      0|     GB|        0.0|             0.0|      1533.95|
|1000003930|Greeting From Ear...|Narrative Film| Film & Video|     USD|2017-11-01| 30000.0|2017-09-02| 2421.0|    failed|     15|     US|      100.0|          2421.0|      30000.0|
|1000004038|      Where is Hank?|Narrative Film| Film

In [7]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: date (nullable = true)
 |-- goal: double (nullable = true)
 |-- launched: date (nullable = true)
 |-- pledged: double (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- usd_pledged: double (nullable = true)
 |-- usd_pledged_real: double (nullable = true)
 |-- usd_goal_real: double (nullable = true)

In [8]:
#describe numeric features
features = ['goal', 'pledged', 'backers', 'usd_pledged_real', 'usd_pledged', 'usd_goal_real']
descriptive_stats = df.describe(features)
descriptive_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+
|summary|              goal|          pledged|           backers| usd_pledged_real|      usd_pledged|     usd_goal_real|
+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+
|  count|            377364|           377547|            377550|           378463|           373760|            378637|
|   mean| 49182.47382447715| 9707.92799362729|107.71595285392664|9060.950684320529|7046.288655902184|  45405.3236660179|
| stddev|1185408.5554960666|95789.98105014741| 945.4115335526068|90996.84734050944|78752.98626499255|1152976.5232637941|
|    min|              0.01|              0.0|                 0|              0.0|              0.0|               0.0|
|    max|             1.0E8|    2.033898627E7|            219382|    2.033898627E7|    2.033898627E7|     1.663613907E8|
+-------+------------------+----

In [9]:
# do we have any rows that are duplicated?
# we do not.
df.count(), df.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(378661, 378661)

# Check for null values

In [10]:
df.rdd.map(
           lambda row: (
                 row['ID']
               , sum([c == None for c in row])
           )
        )\
        .filter(lambda el: el[1] >= 1)\
        .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5091

In [11]:
#count percentage of missing values in each column
for k, v in sorted(
    df.agg(*[
               (fn.count('*') - fn.count(c))
                    .alias(c + '_miss')
               for c in df.columns
           ])
        .collect()[0]
        .asDict()
        .items()
    , key=lambda el: el[1]
    , reverse=True
):
    print(k, v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

usd_pledged_miss 4901
deadline_miss 1297
goal_miss 1297
launched_miss 1114
pledged_miss 1114
backers_miss 1111
usd_pledged_real_miss 198
usd_goal_real_miss 24
state_miss 5
country_miss 5
name_miss 4
currency_miss 4
main_category_miss 3
ID_miss 0
category_miss 0

In [12]:
#Since the rows that contain null values only account for projects whose states 
#are undefined and only account for 1% of the dataset we decided to drop them. 
df = df.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df.rdd.map(
           lambda row: (
                 row['ID']
               , sum([c == None for c in row])
           )
        )\
        .filter(lambda el: el[1] >= 1)\
        .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [14]:
#are there any undefined values in state?
#no there are not
df.rdd.filter(lambda row: row['state'] == "undefined").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

# What are the categories of the cateogrical data?

In [15]:
columns = ['category', 'main_category', 'currency', 'state', 'country']
for i in range(len(columns)):
    new_df = df.groupBy(columns[i]).count()
    print("Number of columns: {}".format(str(new_df.count())))
    new_df.show(22)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of columns: 159
+----------------+-----+
|        category|count|
+----------------+-----+
|       Workshops|  164|
|      Photobooks| 1577|
|         Romance|  190|
|Camera Equipment|  416|
|  Movie Theaters|  230|
|       Festivals|  837|
|      Stationery|  219|
|      Indie Rock| 5610|
|            Food|11475|
|Electronic Music| 2164|
|           Faith| 1086|
|             DIY| 1172|
|       Sculpture| 1799|
|           Latin|  136|
|             Art| 8233|
|           Metal|  716|
|         Candles|  429|
|         Pottery|  100|
|            Apps| 6338|
|        Knitting|  181|
|           Video|  426|
|      Typography|  108|
+----------------+-----+
only showing top 22 rows

Number of columns: 15
+-------------+-----+
|main_category|count|
+-------------+-----+
|         Food|24571|
|          Art|28058|
|      Fashion|22780|
| Film & Video|62416|
|   Publishing|39133|
|       Crafts| 8801|
|       Comics|10787|
|        Games|35196|
|       Design|30019|
|        Music|

In [16]:
#Dropped repeat columns
df = df.drop(*['category', 'pledged', 'goal', 'usd_pledged', 'name', 'currency'])
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+----------+----------+----------+-------+-------+----------------+-------------+
|        ID|main_category|  deadline|  launched|     state|backers|country|usd_pledged_real|usd_goal_real|
+----------+-------------+----------+----------+----------+-------+-------+----------------+-------------+
|1000002330|   Publishing|2015-10-09|2015-08-11|    failed|      0|     GB|             0.0|      1533.95|
|1000003930| Film & Video|2017-11-01|2017-09-02|    failed|     15|     US|          2421.0|      30000.0|
|1000004038| Film & Video|2013-02-26|2013-01-12|    failed|      3|     US|           220.0|      45000.0|
|1000007540|        Music|2012-04-16|2012-03-17|    failed|      1|     US|             1.0|       5000.0|
|1000011046| Film & Video|2015-08-29|2015-07-04|  canceled|     14|     US|          1283.0|      19500.0|
|1000014025|         Food|2016-04-01|2016-02-26|successful|    224|     US|         52375.0|      50000.0|
|1000023410|         Food|2014-12-21|

In [17]:
df = df.withColumn('campaign_length', fn.datediff(fn.col("deadline"), fn.col('launched')))
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+----------+----------+----------+-------+-------+----------------+-------------+---------------+
|        ID|main_category|  deadline|  launched|     state|backers|country|usd_pledged_real|usd_goal_real|campaign_length|
+----------+-------------+----------+----------+----------+-------+-------+----------------+-------------+---------------+
|1000002330|   Publishing|2015-10-09|2015-08-11|    failed|      0|     GB|             0.0|      1533.95|             59|
|1000003930| Film & Video|2017-11-01|2017-09-02|    failed|     15|     US|          2421.0|      30000.0|             60|
|1000004038| Film & Video|2013-02-26|2013-01-12|    failed|      3|     US|           220.0|      45000.0|             45|
|1000007540|        Music|2012-04-16|2012-03-17|    failed|      1|     US|             1.0|       5000.0|             30|
|1000011046| Film & Video|2015-08-29|2015-07-04|  canceled|     14|     US|          1283.0|      19500.0|             56|
|1000014025|    

In [18]:
#describe numeric features
features = ['backers', 'usd_pledged_real', 'usd_goal_real', 'campaign_length']
descriptive_stats = df.describe(features)
descriptive_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------+------------------+------------------+
|summary|           backers| usd_pledged_real|     usd_goal_real|   campaign_length|
+-------+------------------+-----------------+------------------+------------------+
|  count|            373570|           373570|            373570|            373570|
|   mean|106.91540273576572|9139.939011617635| 45956.67955906527|34.494322349225044|
| stddev| 913.2556475886008|91474.23650279021|1160753.4412749212| 66.34077495878944|
|    min|                 0|              0.0|              0.01|                 1|
|    max|            219382|    2.033898627E7|     1.663613907E8|             16739|
+-------+------------------+-----------------+------------------+------------------+

# Getting Rid of Outliers 

## Check Histograms for shape.

In [19]:
#generate bins and count of each bin
histogram_CL = (
    df
    .select('campaign_length')
    .rdd
    .flatMap(lambda record: record)
    .histogram(10)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
%%spark -o hist_CL
#The above Spark magic exports hist_MPG to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_CL =  spark.createDataFrame(
    list(zip(*histogram_CL)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_CL['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of campaign_length'
)

p.vbar(x=labels, top=hist_CL['counts'], width=0.9)

show(p)

Loading BokehJS ...

In [22]:
#generate bins and count of each bin
histogram_backers = (
    df
    .select('backers')
    .rdd
    .flatMap(lambda record: record)
    .histogram(10)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
%%spark -o hist_backers
#The above Spark magic exports hist_MPG to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_backers =  spark.createDataFrame(
    list(zip(*histogram_backers)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_backers['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of backers'
)

p.vbar(x=labels, top=hist_backers['counts'], width=0.9)

show(p)

Loading BokehJS ...

In [25]:
#generate bins and count of each bin
histogram_PR = (
    df
    .select('usd_pledged_real')
    .rdd
    .flatMap(lambda record: record)
    .histogram(10)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
%%spark -o hist_PR
#The above Spark magic exports hist_MPG to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_PR =  spark.createDataFrame(
    list(zip(*histogram_PR)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_PR['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of usd_pledged_real'
)

p.vbar(x=labels, top=hist_PR['counts'], width=0.9)

show(p)

Loading BokehJS ...

# Normalize the data

In [28]:
from pyspark.ml import Pipeline
@udf("double")
def doublify_list(x):
    return round(float(list(x)[0]),3)
    
# Iterating over columns to be scaled
for i in ['backers', 'usd_pledged_real', 'usd_goal_real', 'campaign_length']:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df = pipeline.fit(df).transform(df).withColumn(i+"_Scaled", doublify_list(i+"_Scaled")).drop(*[i+"_Vect"])

print("After Scaling :")
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

After Scaling :
+----------+-------------+----------+----------+--------+-------+-------+----------------+-------------+---------------+--------------+-----------------------+--------------------+----------------------+
|        ID|main_category|  deadline|  launched|   state|backers|country|usd_pledged_real|usd_goal_real|campaign_length|backers_Scaled|usd_pledged_real_Scaled|usd_goal_real_Scaled|campaign_length_Scaled|
+----------+-------------+----------+----------+--------+-------+-------+----------------+-------------+---------------+--------------+-----------------------+--------------------+----------------------+
|1000002330|   Publishing|2015-10-09|2015-08-11|  failed|      0|     GB|             0.0|      1533.95|             59|           0.0|                    0.0|                 0.0|                 0.003|
|1000003930| Film & Video|2017-11-01|2017-09-02|  failed|     15|     US|          2421.0|      30000.0|             60|           0.0|                    0.0|         

In [29]:
columns = ['main_category', 'state', 'country']
for i in range(len(columns)):
    new_df = df.groupBy(columns[i]).count()
    print("Number of columns: {}".format(str(new_df.count())))
    new_df.show(22)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of columns: 15
+-------------+-----+
|main_category|count|
+-------------+-----+
|         Food|24571|
|          Art|28058|
|      Fashion|22780|
| Film & Video|62416|
|   Publishing|39133|
|       Crafts| 8801|
|       Comics|10787|
|        Games|35196|
|        Music|49244|
|       Design|30019|
|  Photography|10750|
|   Technology|32523|
|   Journalism| 4739|
|      Theater|10810|
|        Dance| 3743|
+-------------+-----+

Number of columns: 5
+----------+------+
|     state| count|
+----------+------+
| suspended|  1841|
|    failed|196945|
|  canceled| 38665|
|successful|133324|
|      live|  2795|
+----------+------+

Number of columns: 22
+-------+------+
|country| count|
+-------+------+
|     NL|  2865|
|     MX|  1745|
|     AT|   597|
|     HK|   618|
|     AU|  7824|
|     CA| 14735|
|     GB| 33629|
|     DE|  4166|
|     ES|  2273|
|     US|291456|
|     FR|  2931|
|     CH|   758|
|     SG|   554|
|     IT|  2870|
|     SE|  1756|
|     JP|    40|
|     NZ|  1

In [30]:
clean_rdd = df.rdd
live_campaigns = clean_rdd.filter(lambda row: row['state'] == 'live')
pre_split_rdd = clean_rdd.filter(lambda row: row['state'] != 'live')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
print(live_campaigns.count())
print(pre_split_rdd.count())
print(clean_rdd.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2795
370775
373570

In [32]:
pre_split_df = pre_split_rdd.toDF().withColumn('state', fn.when((fn.col('state') != 'successful'),  'failed').otherwise('successful'))
print('failed: ' + str(pre_split_df.rdd.filter(lambda x: x['state'] == 'failed').count()))
print('successful: ' + str(pre_split_df.rdd.filter(lambda x: x['state'] == 'successful').count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

failed: 237451
successful: 133324

In [33]:
states = ['successful', 'failed']
categories = pre_split_rdd.map(lambda x: x['main_category']).distinct().collect()
countries = pre_split_rdd.map(lambda x: x['country']).distinct().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
state_exprs = [fn.when(fn.col('state') == category, 1).otherwise(0).alias(category)
         for category in states]
categories_exprs = [fn.when(fn.col('main_category') == category, 1).otherwise(0).alias(category)
         for category in categories]
country_exprs = [fn.when(fn.col('country') == category, 1).otherwise(0).alias(category)
         for category in countries]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
pre_split_df = pre_split_df.select(pre_split_df.columns + state_exprs + country_exprs + categories_exprs)
pre_split_df = pre_split_df.drop(*['state', 'main_category', 'country', 'backers', 'usd_pledged_real', 'usd_goal_real', 'campaign_length'])
pre_split_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----------+--------------+-----------------------+--------------------+----------------------+----------+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
|        ID|  deadline|  launched|backers_Scaled|usd_pledged_real_Scaled|usd_goal_real_Scaled|campaign_length_Scaled|successful|failed| NO| CH| FR| HK| IE| NZ| AU| DE| NL| LU| US| GB| IT| JP| CA| MX| ES| DK| BE| SE| SG| AT|Design|Journalism|Food|Theater|Technology|Comics|Publishing|Film & Video|Crafts|Photography|Music|Art|Games|Dance|Fashion|
+----------+----------+----------+--------------+-----------------------+--------------------+----------------------+----------+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----

In [36]:
# Since the dependant variables arent balanced we need to over sample the minority class, in this case sucessful campaigns.
major_df = pre_split_df.filter(fn.col('failed') == 1)
minor_df = pre_split_df.filter(fn.col('successful') == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ratio: 1

In [37]:
a = range(ratio)
# duplicate the minority rows
oversampled_df = minor_df.withColumn("dummy", fn.explode(fn.array([fn.lit(x) for x in a]))).drop('dummy')
oversampled_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

133324

In [38]:
combined_df = major_df.unionAll(oversampled_df)
print(major_df.count()/combined_df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6404180432877081

In [39]:
#The data is now more balanced and close enough to a 1:1 ratio that it shouldnt change the outcome of our results.
#I was not sure what you may have wanted to do with the deadline and launched columns so I left them.
combined_df = combined_df.sort(fn.col("launched")).drop(*['deadline', 'launched', 'failed'])
combined_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+-----------------------+--------------------+----------------------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
|        ID|backers_Scaled|usd_pledged_real_Scaled|usd_goal_real_Scaled|campaign_length_Scaled|successful| NO| CH| FR| HK| IE| NZ| AU| DE| NL| LU| US| GB| IT| JP| CA| MX| ES| DK| BE| SE| SG| AT|Design|Journalism|Food|Theater|Technology|Comics|Publishing|Film & Video|Crafts|Photography|Music|Art|Games|Dance|Fashion|
+----------+--------------+-----------------------+--------------------+----------------------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
|1245461087|           0.0|                    0.

In [40]:
%%spark -o corr
import pyspark.ml.stat as st

features_and_label = feat.VectorAssembler(
    inputCols=combined_df.columns
    , outputCol='features'
)

corr = st.Correlation.corr(
    features_and_label.transform(combined_df), 
    'features', 
    'pearson'
)

print(str(corr.collect()[0][0]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseMatrix([[ 1.00000000e+00,  7.44723786e-04, -4.72123552e-05, ...,
              -1.21208391e-03,  1.81638811e-04, -1.82792368e-03],
             [ 7.44723786e-04,  1.00000000e+00,  7.52863516e-01, ...,
               7.70458723e-02, -8.11821950e-03, -1.18216317e-02],
             [-4.72123552e-05,  7.52863516e-01,  1.00000000e+00, ...,
               4.22571846e-02, -7.15654924e-03, -8.69573906e-03],
             ...,
             [-1.21208391e-03,  7.70458723e-02,  4.22571846e-02, ...,
               1.00000000e+00, -3.24777704e-02, -8.20018847e-02],
             [ 1.81638811e-04, -8.11821950e-03, -7.15654924e-03, ...,
              -3.24777704e-02,  1.00000000e+00, -2.56235132e-02],
             [-1.82792368e-03, -1.18216317e-02, -8.69573906e-03, ...,
              -8.20018847e-02, -2.56235132e-02,  1.00000000e+00]])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
%%local
#this cell exports the correlation matrix to csv
import numpy as np
import pandas as pd
output_np = np.array(corr.iloc[0, 0]['values']).reshape(
    (corr.iloc[0, 0]['numRows'], corr.iloc[0, 0]['numCols']))
pd.DataFrame(output_np).to_csv('corr.csv')

In [42]:
combined_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID', 'backers_Scaled', 'usd_pledged_real_Scaled', 'usd_goal_real_Scaled', 'campaign_length_Scaled', 'successful', 'NO', 'CH', 'FR', 'HK', 'IE', 'NZ', 'AU', 'DE', 'NL', 'LU', 'US', 'GB', 'IT', 'JP', 'CA', 'MX', 'ES', 'DK', 'BE', 'SE', 'SG', 'AT', 'Design', 'Journalism', 'Food', 'Theater', 'Technology', 'Comics', 'Publishing', 'Film & Video', 'Crafts', 'Photography', 'Music', 'Art', 'Games', 'Dance', 'Fashion']

In [43]:
#Drop highly correlated fields
combined_df = combined_df.drop(*['usd_pledged_real_Scaled', 'US'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Transformers

In [44]:
# Split the data into training and test sets (20% held out for testing)
(trainingData, testData) = combined_df.randomSplit([0.8, 0.2], seed=2)
trainingData.show(10)
testData.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+--------------------+----------------------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
|     ID|backers_Scaled|usd_goal_real_Scaled|campaign_length_Scaled|successful| NO| CH| FR| HK| IE| NZ| AU| DE| NL| LU| GB| IT| JP| CA| MX| ES| DK| BE| SE| SG| AT|Design|Journalism|Food|Theater|Technology|Comics|Publishing|Film & Video|Crafts|Photography|Music|Art|Games|Dance|Fashion|
+-------+--------------+--------------------+----------------------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
| 106144|           0.0|                 0.0|                 0.002|         0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0

In [45]:
features = ['backers_scaled',  'usd_goal_real_Scaled', 'campaign_length_Scaled']
quantiles = [0.15, 0.85]

cut_off_points = []

#getting Q1 and Q3
#The third parameter, set to 0.05, is relative target precision to achieve (>= 0).
#If set to zero, the exact quantiles are computed, which could be very expensive. 
for feature in features:
    quants = trainingData.approxQuantile(feature, quantiles, 0.05)
    print(feature, quants)
    IQR = quants[1] - quants[0]
    cut_off_points.append((feature, [
        quants[0] - 1.5 * IQR,
        quants[1] + 1.5 * IQR,
    ]))

cut_off_points = dict(cut_off_points)

outliers = trainingData.select(*['ID'] + [
       (
           (trainingData[f] < cut_off_points[f][0]) |
           (trainingData[f] > cut_off_points[f][1])
       ).alias(f + '_o') for f in features
  ])
outliers.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

backers_scaled [0.0, 0.001]
usd_goal_real_Scaled [0.0, 0.006]
campaign_length_Scaled [0.002, 0.003]
+-------+----------------+----------------------+------------------------+
|     ID|backers_scaled_o|usd_goal_real_Scaled_o|campaign_length_Scaled_o|
+-------+----------------+----------------------+------------------------+
| 106144|           false|                 false|                   false|
| 116600|           false|                 false|                   false|
|1003381|           false|                 false|                   false|
|1024013|           false|                 false|                   false|
|1024208|           false|                 false|                   false|
|1031051|           false|                 false|                   false|
|1057236|           false|                 false|                   false|
|1065827|           false|                 false|                   false|
|1081679|           false|                 false|                   false|


In [46]:
#show the rows containing outliers
with_outliers_flag = trainingData.join(outliers, on='Id')

(
    with_outliers_flag
    .filter('backers_scaled_o OR usd_goal_real_Scaled_o OR campaign_length_Scaled_o')
    .count()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15706

In [47]:
#filter outliers
trainingData_no_outliers = (
    with_outliers_flag
    .filter('!(backers_scaled_o OR usd_goal_real_Scaled_o OR campaign_length_Scaled_o)')
    .select(trainingData.columns)
)
print(trainingData_no_outliers.count())
trainingData_no_outliers.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

280949
+--------+--------------+--------------------+----------------------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
|      ID|backers_Scaled|usd_goal_real_Scaled|campaign_length_Scaled|successful| NO| CH| FR| HK| IE| NZ| AU| DE| NL| LU| GB| IT| JP| CA| MX| ES| DK| BE| SE| SG| AT|Design|Journalism|Food|Theater|Technology|Comics|Publishing|Film & Video|Crafts|Photography|Music|Art|Games|Dance|Fashion|
+--------+--------------+--------------------+----------------------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+----------+----+-------+----------+------+----------+------------+------+-----------+-----+---+-----+-----+-------+
| 1439223|           0.0|                 0.0|                 0.001|         0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|

# Scatter Plots

In [50]:
%%spark -o scatter
#spark magic moves dataframe to local
scatter = (
    trainingData_no_outliers
    .select('backers_Scaled', 'campaign_length_Scaled')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
%%local
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

p = figure(title = 'Relationship between Backers and Campaign Length')
p.xaxis.axis_label = 'Campaign Length'
p.yaxis.axis_label = 'Backers'

p.circle(  list(scatter['campaign_length_Scaled'])
         , list(scatter['backers_Scaled'])
         , fill_alpha=0.2, size=10)

show(p)

Loading BokehJS ...

In [52]:
%%spark -o scatter
#spark magic moves dataframe to local
scatter = (
    trainingData_no_outliers
    .select('backers_scaled', 'usd_goal_real_Scaled')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
%%local
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

p = figure(title = 'Relationship between backers and USD Goal Real')
p.xaxis.axis_label = 'USD Goal Real'
p.yaxis.axis_label = 'Backers'

p.circle(  list(scatter['usd_goal_real_Scaled'])
         , list(scatter['backers_scaled'])
         , fill_alpha=0.2, size=10)

show(p)

Loading BokehJS ...

In [66]:
#generate bins and count of each bin
histogram_goal = (
    trainingData_no_outliers
    .select('usd_goal_real_Scaled')
    .rdd
    .flatMap(lambda record: record)
    .histogram(5)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
%%spark -o hist_goal

import pandas as pd
hist_goal =  spark.createDataFrame(
    list(zip(*histogram_goal)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
%%local

from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_goal['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of goal'
)

p.vbar(x=labels, top=hist_goal['counts'], width=0.9)

show(p)

Loading BokehJS ...

In [57]:
# specify the input and output columns of the vector assembler
assembler = VectorAssembler(inputCols = combined_df.drop('successful').columns,
                                outputCol='features')

#select top 5 features, store in a new column named selected
selector = feat.ChiSqSelector(
    labelCol='successful'
    , numTopFeatures=5
    , outputCol='selected')

#create a multinomial regression object
#DV has 7 classes
logistic_reg = cl.LogisticRegression(
    labelCol='successful'
    , featuresCol=selector.getOutputCol()
    , regParam=0.01
    , elasticNetParam=1.0
)

pipeline_sel = Pipeline(stages=[assembler, selector, logistic_reg])

pModel = pipeline_sel.fit(trainingData_no_outliers)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
metric_df = pModel.transform(testData).select('prediction','successful')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
import pyspark.ml.evaluation as ev

#apply the model to test set
results_logReg = (
    pModel
    .transform(testData)
    .select('successful', 'probability', 'prediction', 'rawPrediction')
)



#evaluate performance of the multinomial model (7 classes)
evaluator = ev.MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='successful')

(
    #F-1 score
    evaluator.evaluate(results_logReg)
    #weighted precision
    , evaluator.evaluate(
        results_logReg
        , {evaluator.metricName: 'weightedPrecision'}
    ) 
    #accuracy
    , evaluator.evaluate(
        results_logReg
        , {evaluator.metricName: 'accuracy'}
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.719053197796185, 0.7838327219838132, 0.7530356179168916)

In [60]:
from pyspark.sql.functions import rand
results_logReg.select('successful', 'probability', 'prediction').orderBy(rand()).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+----------+
|successful|         probability|prediction|
+----------+--------------------+----------+
|         0|[0.82900490885358...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         1|[0.54735504222578...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.82900490885358...|       0.0|
|         1|[0.20932103239319...|       1.0|
|         0|[0.77407065217629...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.87677520548226...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.70771051092908...|       0.0|
|         1|[0.20932103239319...|       1.0|
|         0|[0.70771051092908...|       0.0|
|         0|[0.82900490885358...|       0.0|
|         1|[5.45362412625907...|       1.0|
|         1|[0.63114686051097...|       0.0|
|         

In [61]:
#also assemble and test a random forest classification model
rf_obj = cl.RandomForestClassifier(
    labelCol='successful'
    , featuresCol=selector.getOutputCol()
    , minInstancesPerNode=10
    , numTrees=10
)

pipeline = Pipeline(
    stages=[assembler, selector, rf_obj]
)

pModel = pipeline.fit(trainingData_no_outliers)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
results_rf = (
    pModel
    .transform(testData)
    .select('successful', 'probability', 'prediction', 'rawPrediction')
)

#evaluate performance of the multinomial model (7 classes)
evaluator = ev.MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='successful')

(
    #F-1 score
    evaluator.evaluate(results_rf)
    #weighted precision
    , evaluator.evaluate(
        results_rf
        , {evaluator.metricName: 'weightedPrecision'}
    ) 
    #accuracy
    , evaluator.evaluate(
        results_rf
        , {evaluator.metricName: 'accuracy'})
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.7162641605408824, 0.7874776179326232, 0.7521721532649757)